In [1]:
#pip install NRCLex

In [2]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
sp = "sp500"

In [4]:
df_path = "../data/earning_call_dataframe/df_final_"+sp 
# df_path = "data/df_final"
with open(df_path,"rb") as file :
    df_final = pickle.load(file)
display(df_final)

,id,date,ticker,text,earning_call_quarter
0,4264085,2019-05-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
1,4285738,2019-08-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
2,4309111,2019-11-26,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
3,4325101,2020-02-18,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
4,4349475,2020-05-21,A,Question-and-Answer Session\n \nOperator\n \n[...,2020Q1
...,...,...,...,...,...
1855,4422752,2021-04-30,DRE,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
1856,4442846,2021-07-29,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
1857,4463265,2021-10-28,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
1858,4482287,2022-01-27,DRE,Question-and-Answer Session \n \nOperator\n \n...,2021Q4


# Import datas from yahoo finance

Import html tickers from data/ticker
dir_path = "data/ticker/"
list_tickers = os.listdir(dir_path)
list_tickers.remove('.DS_Store') # for mac
list_tickers.remove('id_to_date')
print(list_tickers)

Test with few stocks
list_tickers = ['AAPL']

In [5]:
# Get the month from which we import stock returns :
    # Q1 : from january (1) to march (3)
    # Q2 : from april (4) to june (6)
    # Q3 : from july (7) to septmber (9)
    # Q4 : from october (10) to december (12)

#df_final['earning_call_quarter'] = pd.to_datetime(df_final['earning_call_quarter'])
#df_final['date'] = pd.to_datetime(df_final['date'])

# Quarters start 1 end
quarter_start = df_final['earning_call_quarter'].iloc[0].quarter
quarter_end = df_final['earning_call_quarter'].iloc[-1].quarter 

# Months start 1 end
month_start = quarter_start*2-1
month_end = quarter_end*3

# Day end : 30 or 31 ?
if month_end in [3,12]:
    day_end = 31
else:
    day_end = 30


In [6]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

# Take a margin of 1 year to calculate abnormal returns
start = datetime.datetime(df_final['earning_call_quarter'].iloc[0].year,month_start,2)
end   = datetime.datetime(df_final['earning_call_quarter'].iloc[-1].year,month_end,day_end)

# Create empty dataframe
stock_final = pd.DataFrame()

# Iterate over each symbol
list_tickers = list(set(list(df_final['ticker'])))
for i in list_tickers:  
    
    # print the symbol which is being downloaded
    print( str(list_tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['ticker']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
display(stock_final)

0 : CAT,1 : BBWI,2 : AAPL,3 : APD,4 : DOV,5 : DAL,6 : CVS,7 : ABT,8 : DFS,9 : AOS,10 : BALL,11 : CB,12 : ALK,13 : CDNS,14 : BBY,15 : ANTM,16 : APA,17 : BR,18 : ADSK,19 : CMG,20 : BDX,21 : AMP,22 : C,23 : D,24 : AMAT,25 : BA,26 : CBOE,27 : DLR,28 : ADP,29 : AES,30 : BLK,31 : BEN,32 : CAH,33 : CPT,34 : AVY,35 : CLX,36 : CCI,37 : CHTR,38 : BRO,39 : A,40 : BIIB,41 : BAC,42 : BKNG,43 : DRE,44 : AKAM,45 : BSX,46 : CBRE,47 : ADM,48 : CDAY,49 : CINF,50 : CME,51 : DG,52 : CF,53 : CE,54 : CHRW,55 : AME,56 : ARE,57 : DHR,58 : CZR,59 : ATO,60 : ANET,61 : BKR,62 : AON,63 : DIS,64 : CMS,65 : AXP,66 : BXP,67 : AEP,68 : ALGN,69 : DD,70 : CSX,71 : APH,72 : CTSH,73 : AEE,74 : AZO,75 : AVB,76 : ALB,77 : ANSS,78 : DHI,79 : CMI,80 : AAL,81 : APTV,82 : BMY,83 : DLTR,84 : ALL,85 : COF,86 : AIG,87 : DGX,88 : ABMD,89 : DE,90 : ABBV,91 : AIZ,92 : CI,93 : CMA,94 : AMZN,95 : AFL,96 : CHD,97 : BK,98 : CDW,99 : CL,

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2019-01-02,124.029999,127.879997,123.000000,126.379997,115.741951,4783200,CAT
2019-01-03,125.309998,125.500000,120.440002,121.510002,111.281891,6165800,CAT
2019-01-04,124.169998,128.589996,123.699997,128.149994,117.362968,5456600,CAT
2019-01-07,128.589996,129.600006,126.699997,128.229996,117.436234,4653400,CAT
2019-01-08,129.899994,131.500000,128.229996,129.770004,118.846619,4202600,CAT
...,...,...,...,...,...,...,...
2022-03-24,73.660004,74.010002,73.169998,73.650002,73.220581,3539500,CL
2022-03-25,73.910004,75.059998,73.779999,74.779999,74.343994,4401000,CL
2022-03-28,74.839996,75.419998,74.839996,75.389999,74.950439,3646200,CL


In [7]:
# Add quarter column
stock_final['date']=stock_final.index
stock_final['quarter'] = stock_final['date'].dt.to_period('Q')
# Calculate market capitalization
stock_final['market_cap']=stock_final['Adj Close']*stock_final['Volume']
# Drop useless columns
stock_final=stock_final.drop(columns=['Open', 'High','Low','Close','date','Adj Close','Volume'])

# Sum of 100 stocks for each quarter
stock_final = stock_final.groupby(['quarter']).sum()
stock_final['percent_change']=stock_final['market_cap'].pct_change()

stock_final = stock_final.iloc[1:,:]  # remove first row
stock_final.index = stock_final.index.to_timestamp()
display(stock_final)

,market_cap,percent_change
quarter,,
2019-04-01,2.359815e+12,-0.053806
2019-07-01,2.220580e+12,-0.059002
2019-10-01,2.352039e+12,0.059200
2020-01-01,3.722236e+12,0.582557
2020-04-01,3.887510e+12,0.044402
2020-07-01,4.117948e+12,0.059277
2020-10-01,3.796502e+12,-0.078060
2021-01-01,3.833756e+12,0.009813
2021-04-01,3.513645e+12,-0.083498


# Text analysis

In [8]:
# Preprocessing
texts = df_final['text']
n = len(texts)

In [9]:
#N=len(texts)

for i in range(n):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        con = np.vstack((con, score))

In [10]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)
df[column]=df[column].astype(float)

# Earning call informations
df['id'] = df_final['id']
df['quarter'] = df_final['earning_call_quarter']
df['ticker'] = df_final['ticker']

# Calculate change in sentiments from one quarter to the other
df_change = df
df_change[column] = df[column].pct_change()
# Mean over each quarter
df_change = df_change.groupby(['quarter']).mean()
# Remove first row
df_change=df_change.iloc[1:,:]
# Remplace Nan with zeros
df_change = df_change.fillna(0)
df_change.index = (df_change.index).to_timestamp()
display(df_change)

# Merge dataframes
df_ols = pd.merge(stock_final, df_change, left_index=True, right_index=True)
# Drop first row
#df_change=df_change.drop(df_change.groupby('ticker').head(1).index, axis=0)
display(df_ols)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
quarter,,,,,,,,,,
2019-04-01,-0.026106,0.172643,0.151611,-0.324919,0.377327,-0.083514,-0.022175,-0.103786,0.261362,0.064781
2019-07-01,-0.009045,0.029816,-0.459479,0.081042,0.243965,0.146399,0.218529,0.376327,-0.036194,0.141862
2019-10-01,0.108000,0.085701,1.105201,1.192917,-0.295293,0.195326,-0.183799,0.136808,0.260155,-0.248805
2020-01-01,0.952888,-0.373987,0.115936,-0.062614,-0.110076,0.249182,-0.071187,0.074605,-0.487982,-0.042499
2020-04-01,-0.279295,0.333088,-0.637577,-0.295863,0.201175,-0.246159,0.064171,-0.063275,0.293573,-0.030061
2020-07-01,-0.383154,-0.141768,1.525441,-0.442918,0.112742,0.165588,0.153083,0.089848,0.124837,0.289876
2020-10-01,-0.275954,-0.012004,-0.447683,0.052033,-0.032587,-0.316179,-0.091975,-0.249066,-0.144799,-0.078087
2021-01-01,0.546988,0.294170,0.117269,0.675903,0.081611,-0.011647,0.180931,0.214423,0.017513,-0.033713
2021-04-01,-0.425111,-0.113414,-0.731221,-0.435563,-0.235012,0.042981,-0.206340,-0.021643,0.143630,-0.165104


,market_cap,percent_change,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
quarter,,,,,,,,,,,,
2019-04-01,2.359815e+12,-0.053806,-0.026106,0.172643,0.151611,-0.324919,0.377327,-0.083514,-0.022175,-0.103786,0.261362,0.064781
2019-07-01,2.220580e+12,-0.059002,-0.009045,0.029816,-0.459479,0.081042,0.243965,0.146399,0.218529,0.376327,-0.036194,0.141862
2019-10-01,2.352039e+12,0.059200,0.108000,0.085701,1.105201,1.192917,-0.295293,0.195326,-0.183799,0.136808,0.260155,-0.248805
2020-01-01,3.722236e+12,0.582557,0.952888,-0.373987,0.115936,-0.062614,-0.110076,0.249182,-0.071187,0.074605,-0.487982,-0.042499
2020-04-01,3.887510e+12,0.044402,-0.279295,0.333088,-0.637577,-0.295863,0.201175,-0.246159,0.064171,-0.063275,0.293573,-0.030061
2020-07-01,4.117948e+12,0.059277,-0.383154,-0.141768,1.525441,-0.442918,0.112742,0.165588,0.153083,0.089848,0.124837,0.289876
2020-10-01,3.796502e+12,-0.078060,-0.275954,-0.012004,-0.447683,0.052033,-0.032587,-0.316179,-0.091975,-0.249066,-0.144799,-0.078087
2021-01-01,3.833756e+12,0.009813,0.546988,0.294170,0.117269,0.675903,0.081611,-0.011647,0.180931,0.214423,0.017513,-0.033713
2021-04-01,3.513645e+12,-0.083498,-0.425111,-0.113414,-0.731221,-0.435563,-0.235012,0.042981,-0.206340,-0.021643,0.143630,-0.165104


# OLS

In [11]:
# ols
result = sm.ols(formula="percent_change ~ anger+anticipation+disgust+fear+joy+negative+positive+sadness+surprise+trust", data=df_ols).fit()
result.summary()

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         percent_change   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                 -1.164
Method:                 Least Squares   F-statistic:                    0.4083
Date:                Fri, 17 Jun 2022   Prob (F-statistic):              0.851
Time:                        17:44:23   Log-Likelihood:                 13.928
No. Observations:                  12   AIC:                            -5.857
Df Residuals:                       1   BIC:                           -0.5230
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0325      0.146      0.223      0.860      -1.822       1.888
anger            0.6153      1.111      0.554      0.678     -13.497      14.728
anticipation    -0.6866      2.402     -0.286      0.823     -31.212      29.839
disgust          0.0017      0.294      0.006      0.996      -3.736       3.739
fear            -0.1398      0.355     -0.393      0.761      -4.656       4.376
joy             -0.3482      3.040     -0.115      0.927     -38.981      38.284
negative        -0.7601      2.901     -0.262      0.837     -37.615      36.095
positive        -0.3154      6.192     -0.051      0.968     -78.989      78.358
sadness          0.4729      2.168      0.218      0.863     -27.072      28.018
surprise         0.5670      1.112      0.510      0.700     -13.559      14.693
trust            0.4832      8.814      0.055      0.965    -111.507     112.474
==============================================================================
Omnibus:                        1.302   Durbin-Watson:                   0.575
Prob(Omnibus):                  0.522   Jarque-Bera (JB):                1.025
Skew:                           0.563   Prob(JB):                        0.599
Kurtosis:                       2.117   Cond. No.                         190.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""